In [45]:
#imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


import helper


In [46]:
#Load EEG data - preprocessed
import mne
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib qt


#Subject numbers and experiment
sub = "sub-28"
exp = "fixthemix"

fname = f"derivatives/eegprep/{sub}/{sub}_task-{exp}_eegprep.vhdr"

raw = mne.io.read_raw_brainvision(fname, preload=True)
events, event_dict = mne.events_from_annotations(raw)
#start of songs in sample numbers
song_starts = np.array(events)[events[:,2] == 10001][2:,0]
press_starts = []
press_starts = events[2:,0]

print(event_dict)
#set sample rate
sample_rate = 250
#raw.plot()

Extracting parameters from derivatives/eegprep/sub-28/sub-28_task-fixthemix_eegprep.vhdr...
Setting channel info structure...
Reading 0 ... 465922  =      0.000 ...  1863.688 secs...
Used Annotations descriptions: ['New Segment/', 'Stimulus/1', 'Stimulus/11', 'Stimulus/12', 'Stimulus/13', 'Stimulus/15', 'Stimulus/2', 'Stimulus/21', 'Stimulus/22', 'Stimulus/23', 'Stimulus/25', 'Stimulus/3', 'Stimulus/31', 'Stimulus/32', 'Stimulus/33', 'Stimulus/35', 'Stimulus/41', 'Stimulus/42', 'Stimulus/43', 'Stimulus/45', 'Time 0/']
{'New Segment/': 99999, 'Stimulus/1': 10001, 'Stimulus/11': 10002, 'Stimulus/12': 10003, 'Stimulus/13': 10004, 'Stimulus/15': 10005, 'Stimulus/2': 10006, 'Stimulus/21': 10007, 'Stimulus/22': 10008, 'Stimulus/23': 10009, 'Stimulus/25': 10010, 'Stimulus/3': 10011, 'Stimulus/31': 10012, 'Stimulus/32': 10013, 'Stimulus/33': 10014, 'Stimulus/35': 10015, 'Stimulus/41': 10016, 'Stimulus/42': 10017, 'Stimulus/43': 10018, 'Stimulus/45': 10019, 'Time 0/': 10020}


In [47]:
#Load FLAC Audio in
import pyflac
import scipy.io.wavfile as wav
import scipy.signal as sig
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

aname = f"derivatives/audio/{sub}/{sub}_task-{exp}_aud.flac"
decoder = pyflac.FileDecoder(aname, "temp.wav")
samp_rate, audio = wav.read("temp.wav")
print(audio.shape)
print(raw.get_data().shape)

#Resample audio to EEG sample rate and get audio envelope
audio = sig.resample(audio, raw.get_data().shape[1])
audio = np.abs(sig.hilbert(audio.T))
audio = np.average(audio, axis=0)
print(audio)
print(np.max(audio), np.min(audio))

#normalize audio
scaler = StandardScaler()
audio = scaler.fit_transform(audio.reshape(-1,1)).reshape(1, -1)
print(audio)
print(np.max(audio), np.min(audio))
print(np.mean(audio), np.std(audio))

audio = np.atleast_2d(audio)

(82188285, 2)
(31, 465923)
[3.89786261 2.16101743 3.89697985 ... 2.16272319 3.89874611 2.16186872]
37195.47015751086 2.014702518637015
[[-0.81162915 -0.81207714 -0.81162937 ... -0.8120767  -0.81162892
  -0.81207692]]
8.781329033616105 -0.8121148767531039
-1.7568251225967283e-17 1.0000000000000002


In [77]:
#SPLIT BY SEGMENT

#Split audio and eeg up into their corresponding songs
#note that the 1st elements are the whitespace before the first song starts

num_segments = 300
seconds = 5

split_eeg = raw.get_data()

#split data into 300 segments
times = np.linspace(song_starts[1], raw.get_data().shape[1], num_segments, dtype = int)
#each segment has a bound of "seconds" seconds before and after the segment
split_audio, split_eeg = helper.split_events(audio, split_eeg, times, sample_rate, seconds)

fs = sample_rate

labels = [f'song{i}' for i in range(1, len(split_audio))]
labels_train, labels_test = train_test_split(labels, train_size=0.7, test_size=0.3, random_state=5)
labels_test, labels_val = train_test_split(labels_test, train_size=0.5, test_size=0.5, random_state=5)

#X and Y are dictionaries so that the ordering of the corresponding segments can
#be maintained
X = {}
Y = {}
for i in range(1,len(split_audio)):
    X[f'song{i}'] = split_audio[i][0]

    Y[f'song{i}'] = split_eeg[i]

size = Y['song3'].T.shape

X_test, X_train, X_val, Y_test, Y_train, Y_val = [],[],[],[],[],[]

for i in labels_train:
    if(Y[i].T.shape == size):
        Y_train.append(scaler.fit_transform(X[i].reshape(-1,1)).reshape(1,-1))
        X_train.append(scaler.fit_transform(Y[i].T))

for i in labels_val:
    if(Y[i].T.shape == size):
        Y_val.append(scaler.fit_transform(X[i].reshape(-1,1)).reshape(1,-1))
        X_val.append(scaler.fit_transform(Y[i].T))

for i in labels_test:
    if(Y[i].T.shape == size):
        Y_test.append(scaler.fit_transform(X[i].reshape(-1,1)).reshape(1,-1))
        X_test.append(scaler.fit_transform(Y[i].T))

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_val = np.array(X_val)
Y_val = np.array(Y_val)




In [114]:
#size of latent space dimensions 
latent = 10

input_shape = X_train.shape[1:]
output_shape = Y_train.shape[1:]
print(input_shape)
print(output_shape)

print(len(X_train))
print(len(X_test))
print(len(X_val))

#norm_layer.adapt(data=np.array(X_train))


#Autoencoder model definition
class Autoencoder(Model):
    def __init__(self, latent_dim, input_shape, output_shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=self.input_shape),
            layers.Conv1D(64, 3, strides=2, padding='same', activation='relu'),
            layers.MaxPooling1D(),
            layers.Dropout(0.25),
            layers.Conv1D(32, 3, strides=2, padding='same', activation='relu'),\
            layers.MaxPooling1D(),
            layers.Dropout(0.25),
            layers.Conv1D(16, 3, strides=2, padding='same', activation='relu'),\
            layers.MaxPooling1D(),
            layers.Dropout(0.25),
            layers.Flatten(),
            layers.Dense(self.latent_dim),
        ])

        self.decoder = tf.keras.Sequential([
            #layers.Dense(self.output_shape[1]*20),
            layers.Reshape((-1, 2)),
            layers.Conv1DTranspose(20, 3, strides=2, padding='same', activation='relu'),
            layers.UpSampling1D(5),
            layers.Conv1DTranspose(10, 3, strides=2, padding='same', activation='relu'),
            layers.UpSampling1D(5),
            layers.Conv1DTranspose(5, 3, strides=2, padding='same', activation='relu'),
            layers.Conv1DTranspose(1, 3, strides=2, padding='same', activation='relu'),
            layers.Conv1D(25, 3, strides=2, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(self.output_shape[1]),
            layers.Reshape(self.output_shape)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def predict(self, data, labels):
        reconstructions = model(data).numpy()
        loss = losses.mse(reconstructions, labels)
        return reconstructions, loss

    
model = Autoencoder(latent, input_shape = input_shape, output_shape = output_shape)
model.encoder.summary()
model.compile(optimizer="Adam", loss=losses.MeanSquaredError())

history = model.fit(X_train, Y_train,
          epochs=20,
          validation_data=(X_val, Y_val),
          )

model.decoder.summary()


(2500, 31)
(1, 2500)
208
44
45


Model: "sequential_157"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_338 (Conv1D)             │ (None, 1250, 64)       │         6,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_261               │ (None, 625, 64)        │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_101 (Dropout)           │ (None, 625, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_339 (Conv1D)             │ (None, 313, 32)        │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_262               │ (None, 156, 32)        │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_102 (Dropout)           │ (None, 156, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_340 (Conv1D)             │ (None, 78, 16)         │         1,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_263               │ (None, 39, 16)         │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_103 (Dropout)           │ (None, 39, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_135 (Flatten)           │ (None, 624)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_240 (Dense)               │ (None, 10)             │         6,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,994 (78.10 KB)

 Trainable params: 19,994 (78.10 KB)

...

Model: "sequential_158"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_241 (Dense)               │ (None, 625)            │         6,875 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_242 (Dense)               │ (None, 1250)           │       782,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_243 (Dense)               │ (None, 2500)           │     3,127,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_147 (Reshape)           │ (None, 1, 2500)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,916,875 (14.94 MB)

 Trainable params: 3,916,875 (14.94 MB)

 Non-trainable params: 0 (0.00 B)

In [91]:
X_test = np.array(X_test)
Y_test = np.array(Y_test)
display_Y = model(X_test[0].reshape(1, X_test[0].shape[0], X_test[0].shape[1])).numpy().reshape(1,-1)
true_Y = Y_test[0]

print(true_Y.shape)
print(display_Y.shape)
display_scale = StandardScaler().fit(true_Y)
display_Y = display_scale.transform(display_Y)

pred, loss = model.predict(X_test, Y_test)

print(loss)

fig, axs = plt.subplots(3,1)
axs[0].plot(display_Y[0])
axs[0].set_title("Reconstructed Audio Envelope")
axs[1].plot(true_Y[0])
axs[1].set_title("True Audio Envelope")
axs[2].plot(-1*display_Y[0], color="Red", label="Reconstructed (Flipped)")
axs[2].plot(true_Y[0], label="True", alpha=0.5, color="Green")
axs[2].set_title("True Audio Envelope vs Reconstructed")
fig.legend()
plt.show()

(1, 2500)
(1, 2500)
tf.Tensor(
[[1.17874232]
 [0.88453531]
 [1.16197678]
 [1.06962984]
 [1.06909225]
 [1.10079861]
 [1.1150897 ]
 [0.95318884]
 [1.00956686]
 [1.10303659]
 [0.99703505]
 [1.03181249]
 [1.01982038]
 [1.12149397]
 [0.90419176]
 [1.00203959]
 [1.29085632]
 [1.07136301]
 [0.99576511]
 [1.087238  ]
 [1.01063237]
 [0.96388514]
 [0.96291417]
 [1.15422213]
 [1.16544797]
 [0.99288187]
 [0.99156986]
 [0.89529545]
 [0.91441977]
 [0.9940612 ]
 [1.02272036]
 [0.73097921]
 [0.98376906]
 [1.09316868]
 [0.94392749]
 [1.18658056]
 [1.06212826]
 [1.11449181]
 [1.01161385]
 [1.06247554]
 [1.25189378]
 [1.12036441]
 [1.00828466]
 [0.98714215]], shape=(44, 1), dtype=float64)


In [92]:
plt.figure()
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()